In [31]:
from langchain import PromptTemplate        
from langchain.chains import RetrievalQA                
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter      #for creating chunks
from langchain.prompts import PromptTemplate
from langchain.llms import ctransformers 
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec

In [2]:
# PINCONE_API_KEY="dfad884a-1a06-493c-850a-1de28db1b2dc"
# PINCONE_API_ENV=""
pc = Pinecone(api_key="dfad884a-1a06-493c-850a-1de28db1b2dc")

index_name = "mchatbot"

if index_name not in pc.list_indexes().names():
  pc.create_index(
  name="mchatbot",
  dimension=384,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
) 

In [3]:
# Data extraction from pdf

def load_pdf(data):
    loader= DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents= loader.load()

    return documents

In [4]:
extracted_data = load_pdf("data/")

In [5]:
# extracted_data

In [6]:
# Creating text Chunks so that the entire pdf is splitted into smaller texts

def text_split(extracted_data):
    # keeping size of each chunk to be 500 with overlap b/w chunks as 20
    text_splitter= RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap = 20)
    text_chunks= text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks=text_split(extracted_data)
print("No of chunks formed:", len(text_chunks))

No of chunks formed: 10484


In [8]:
# text_chunks

In [20]:
# Convert these chunks made into vector

# Download the embeddig model

def download_hugging_face_embedding():
    embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [24]:
embeddings= download_hugging_face_embedding()

In [25]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [32]:
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings_test = model.encode(sentences)
print(embeddings_test)

[[ 6.76568821e-02  6.34959117e-02  4.87130992e-02  7.93049559e-02
   3.74480709e-02  2.65276246e-03  3.93749475e-02 -7.09846709e-03
   5.93614057e-02  3.15369777e-02  6.00980185e-02 -5.29051945e-02
   4.06067632e-02 -2.59308536e-02  2.98428200e-02  1.12689298e-03
   7.35148937e-02 -5.03819436e-02 -1.22386597e-01  2.37028506e-02
   2.97265016e-02  4.24768664e-02  2.56337766e-02  1.99518842e-03
  -5.69190569e-02 -2.71598101e-02 -3.29035372e-02  6.60248622e-02
   1.19007103e-01 -4.58791144e-02 -7.26214573e-02 -3.25839929e-02
   5.23412935e-02  4.50552665e-02  8.25302675e-03  3.67024131e-02
  -1.39415450e-02  6.53918609e-02 -2.64272708e-02  2.06391443e-04
  -1.36643369e-02 -3.62809859e-02 -1.95043422e-02 -2.89738253e-02
   3.94270308e-02 -8.84091035e-02  2.62428750e-03  1.36714010e-02
   4.83062789e-02 -3.11565902e-02 -1.17329165e-01 -5.11689968e-02
  -8.85287747e-02 -2.18962301e-02  1.42986113e-02  4.44167592e-02
  -1.34815220e-02  7.43392929e-02  2.66382508e-02 -1.98762640e-02
   1.79191

In [30]:
query_result = embeddings.embed_query("Hello World")
print("Length of embedding:", len(query_result))

AttributeError: 'numpy.ndarray' object has no attribute 'embed_query'